#### The requests package is installed

In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


#### The BeautifulSoup and Requests library are imported

In [267]:
from bs4 import BeautifulSoup
import requests
import time
from time import sleep
import pandas as pd
import numpy as np
pd.options.display.max_columns=None

In [306]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------- 382.5/382.5 kB 1.1 MB/s eta 0:00:00
     ------------------------------------- 269.1/269.1 kB 1.4 MB/s eta 0:00:00


In [264]:
url_list=[]

def get_url():
    
    global base_url
    
    base_url = 'https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=1&gk=GIWEDR&cme=false'
    
    for i in range(2,3):
            url_split=base_url.split('pg=')
            url_part2 =url_split[1]
            url_next_page=url_split[0] + 'pg=' + url_part2.replace('1',str(i))
            
            url_list.append(url_next_page)
            
    url_list.insert(0,base_url) 
    
    for i in url_list:
        print(i)

In [265]:
def get_guitar_details():
    
    global Guitar_Feature_Type_list
    global Guitar_Feature_Name_list
    global soup
    global soup2
    global guitar_details
    
    
    get_url()
    
    guitar_details=[]
    
    
    
    for i in url_list:
        
        result=requests.get(i, timeout=5, headers={'Connection':'close'})

        print('Response Status Code:  ' + str(result.status_code))

        src=result.text
        
        result.close()

        soup= BeautifulSoup(src,'lxml') 

        dread_guitars=soup.find('div',class_ = "js-search")

        collection=dread_guitars.find_all('div', class_="fx-product-list-entry")

        


        for guitar in collection:

                Product=guitar.find('a', class_='product__content')

                Manufacturer=Product.find('span', class_='title__manufacturer').text
                
                Model=Product.find('span', class_='title__name').text
                
                Price_unclean=Product.find('div', class_= 'product__price').text

                if Price_unclean is not None:
                    
                    Price_unclean.strip()

                    Price=Price_unclean.strip().lstrip('£')
                
                else:
                    
                    Price= None
                

                try:
                    Product_availability_unclean=Product.find('div',class_='product__availability fx-availability fx-availability--in-stock').text

                    Product_availability=Product_availability_unclean.strip()

                except:
                    Product_availability= None
                
                
                
                link_part2=guitar.find('a', class_='product__content')['href']
                
                if link_part2 is not None:
                    
                        link_product='https://www.thomann.de/gb/' + link_part2 


                        result2=requests.get(link_product,timeout=5, headers={'Connection':'close'})

                        src2=result2.text

                        result2.close()


                        soup2=BeautifulSoup(src2,'lxml')

                        for i in soup2.find_all('div',class_='keyfeature keyfeature--hidden fx-collapsible__content'):
                            i.decompose()

                        for i in soup2.find_all('div',class_='keyfeature keyfeature--toggle-trigger'):
                            i.decompose()



                        try:

                            guitar_info=soup2.find('div', class_='fx-grid__col fx-col--12 fx-col--lg-4')
                            guitar_rank=guitar_info.find('span', class_='meta-box__value').text.strip()


                        except:

                            guitar_rank=None


                        parent=soup2.find('div', class_='badges')

                        try:

                            for i,j in enumerate(parent):
                                if i==3:
                                    years_warranty=j.text

                            index=years_warranty.find('-')
                            years_warranty=years_warranty[index-1]

                        except:

                            years_warranty=None



                        key_features=soup2.find('div', class_='keyfeatures')

                        try:

                                feature_type=key_features.find_all('span', class_='keyfeature__label')
                                feature_name=key_features.find_all('span', class_ ='fx-text fx-text--bold fx-text--no-margin')

                                Guitar_Feature_Type_list=[]
                                Guitar_Feature_Name_list=[]

                                for i in feature_type:
                                    x=i.text
                                    Guitar_Feature_Type_list.append(x)

                                for j in feature_name:
                                    y=j.text
                                    Guitar_Feature_Name_list.append(y)

                        except:

                                Guitar_Feature_Type_list.append(None)
                                Guitar_Feature_Name_list.append(None)


                        entry={}

                        for i,j in zip(Guitar_Feature_Type_list,Guitar_Feature_Name_list):

                                entry['Manufacturer']= Manufacturer
                                entry['Model']= Model
                                entry['Price']= Price
                                entry['Product_availability']= Product_availability
                                entry['guitar_rank']= guitar_rank
                                entry['Years_Warranty']=years_warranty

                                entry[i.upper()]=j

                        guitar_details.append(entry)

                        time.sleep(3)
                
                else:
                        print('Product Link not available')
                        
                        entry={}
                        entry['Manufacturer']= Manufacturer
                        entry['Model']= Model
                        entry['Price']= Price
                        entry['Product_availability']= Product_availability
                        
                        guitar_details.append(entry)
                        
                    

In [266]:
get_guitar_details()

https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=1&gk=GIWEDR&cme=false
https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=2&gk=GIWEDR&cme=false
Response Status Code:  200
Response Status Code:  200


In [268]:
df3=pd.json_normalize(guitar_details)

In [269]:
df3.shape

(100, 30)

In [452]:
df=df3.copy()

In [453]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Manufacturer            100 non-null    object
 1   Model                   100 non-null    object
 2   Price                   100 non-null    object
 3   Product_availability    81 non-null     object
 4   guitar_rank             100 non-null    object
 5   Years_Warranty          100 non-null    object
 6   AVAILABLE SINCE         100 non-null    object
 7   ITEM NUMBER             100 non-null    object
 8   SALES UNIT              100 non-null    object
 9   TOP                     99 non-null     object
 10  BACK AND SIDES          99 non-null     object
 11  CUTAWAY                 99 non-null     object
 12  FRETBOARD               99 non-null     object
 13  NUT WIDTH IN MM         99 non-null     object
 14  FRETS                   99 non-null     object
 15  PICKUPS

In [454]:
df.head()

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
0,Fender,CD-60S Black WN,145,In stock,60,3,January 2019,455910,1 piece(s),"Spruce, Solid",Mahogany,No,Walnut,"43,00 mm",20,No,Black,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Harley Benton,D-120CE BK,75,In stock,177,3,January 2003,157819,1 piece(s),Spruce,"Sapelli, Mahogany",Yes,Roseacer,"42,50 mm",20,Yes,Black,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Harley Benton,CLD-60SCE TB,111,In stock,61,3,December 2020,500960,1 piece(s),"Spruce, Solid",Mahogany,Yes,Roseacer,"43,00 mm",20,Yes,Blue,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fender,CD-60 NA V3,111,In stock,29,3,September 2006,190655,1 piece(s),Spruce,Mahogany,No,Walnut,"43,00 mm",20,No,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Harley Benton,D-120CE NT Bundle,85,In stock,16,3,June 2017,416052,1 piece(s),Spruce,"Sapelli, Mahogany",Yes,Roseacer,"42,50 mm",20,Yes,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [455]:
df.tail()

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
95,Recording King,RD-328 Aged Adirondack,655,In stock,106,3,May 2020,493963,1 piece(s),"Spruce, Solid",Solid Rosewood,No,Rosewood,"42,90 mm",20,No,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Cort,Earth 100 NS,239,None,114,3,March 2018,434922,1 piece(s),"Spruce, Solid",Mahogany,No,Ovangkol,"43,00 mm",20,No,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Guild,D-260CE Deluxe NAT,329,In stock,134,3,March 2019,457747,1 piece(s),Sitka Spruce solid,Ebony,Yes,Pau Ferro,"43,00 mm",20,Yes,"Natural, High Gloss",No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Harley Benton,CLD-40SM-CE SolidWood,319,In stock,109,3,June 2019,461137,1 piece(s),"Mahogany, Solid",Solid Mahogany,Yes,Ebonol,"43,00 mm",20,Yes,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,Guild,D-120CE Nat Westerly,599,None,103,3,May 2015,361964,1 piece(s),"Mahogany, Solid",Solid Mahogany,Yes,Rosewood,"44,50 mm",20,Yes,Natural,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [456]:
df.isnull().sum()

Manufacturer               0
Model                      0
Price                      0
Product_availability      19
guitar_rank                0
Years_Warranty             0
AVAILABLE SINCE            0
ITEM NUMBER                0
SALES UNIT                 0
TOP                        1
BACK AND SIDES             1
CUTAWAY                    1
FRETBOARD                  1
NUT WIDTH IN MM            1
FRETS                      1
PICKUPS                    2
COLOUR                     0
CASE                       1
INCL. GIGBAG               1
DESIGN                    99
POWER IN WATTS            99
REVERB                    99
EFFECTS                   99
HEADPHONE CONNECTION      99
INCL. CASE / GIGBAG       99
INCL. CABEL               99
INCL. PICKS               99
INCL. TUNER               99
INCL. STRAP               99
INCL. EDUCATIONAL BOOK    99
dtype: int64

In [457]:
df[df['PICKUPS'].isnull()==True]

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
76,Epiphone,Slash J-45 November Burst,694,None,58,3,December 2021,518609,1 piece(s),"Spruce, Solid",Mahogany,No,indian Laurel,"43,00 mm",20,NaN,Sunburst,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Harley Benton,HBD120CEBK Bundle,103,In stock,43,3,March 2007,111395,1 piece(s),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black,NaN,NaN,Dreadnought,0 W,No,No,No,Yes,No,Yes,Yes,No,Yes


In [458]:
df['PICKUPS'].fillna('No',inplace=True)

In [459]:
df[df['FRETBOARD'].isnull()==True]

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
92,Harley Benton,HBD120CEBK Bundle,103,In stock,43,3,March 2007,111395,1 piece(s),NaN,NaN,NaN,NaN,NaN,NaN,No,Black,NaN,NaN,Dreadnought,0 W,No,No,No,Yes,No,Yes,Yes,No,Yes


In [460]:
df.dropna(axis=0, subset= 'FRETBOARD', inplace=True)

In [461]:
column_list=['SALES UNIT','DESIGN','POWER IN WATTS','REVERB','EFFECTS','HEADPHONE CONNECTION','INCL. CASE / GIGBAG','INCL. CABEL','INCL. PICKS','INCL. TUNER','INCL. STRAP','INCL. EDUCATIONAL BOOK']

for i in column_list:
    if i in list(df.columns):
        df.drop(i, axis=1, inplace=True)

In [462]:
for column in list(df.columns):
    df.rename(columns = {column : column.upper()} , inplace = True)

In [463]:
df.rename(columns = {'PRICE in EURO' : 'PRICE'} , inplace = True)

In [464]:
df['PRODUCT_AVAILABILITY'].fillna(value='Unavailable', inplace=True)

In [465]:
df['YEARS_WARRANTY'].unique()

array(['3'], dtype=object)

In [466]:
df['MANUFACTURER'].unique()

array(['Fender', 'Harley Benton', 'Epiphone', 'Takamine', 'DAngelico',
       'Gibson', 'Squier', 'Taylor', 'Guild', 'Ibanez', 'Seagull', 'Cort',
       'Yamaha', 'Recording King'], dtype=object)

In [467]:
df['TOP'].unique()

array(['Spruce, Solid', 'Spruce', 'Mahogany', 'Cedar', 'Mahogany, Solid',
       'Cedar, Solid', 'Ebony', 'Coa solid', 'Sitka Spruce solid'],
      dtype=object)

In [468]:
df['BACK AND SIDES'].unique()

array(['Mahogany', 'Sapelli, Mahogany', 'Maple', 'Walnut', 'Koa, Solid',
       'Solid Mahogany', 'Walnut, Solid', 'Nato', 'Ziricote', 'Rosewood',
       'Sapele', 'Cherry', 'Solid Rosewood', 'Ovangkol',
       'Ovangkol, Solid', 'Maple, Solid', 'Okume', 'Basswood', 'Ebony'],
      dtype=object)

In [469]:
df['FRETBOARD'].unique()

array(['Walnut', 'Roseacer', 'Pau Ferro', 'Rosewood', 'Laurel',
       'Ovangkol', 'Ebony', 'Merbau', 'Maple', 'indian Laurel', 'Nandu',
       'Blackwood', 'Ebonol'], dtype=object)

In [470]:
df['FRETS'].unique()

array(['20', '643', '21'], dtype=object)

In [471]:
df['FRETS']=df['FRETS'].astype('int')

In [472]:
for index,value in enumerate(df['FRETS']):
    if value >= 24 :
        df.drop(index, axis=0, inplace=True)

In [473]:
df['COLOUR'].unique()

array(['Black', 'Blue', 'Natural', 'Sunburst', 'Natural Mahogany',
       'Sunburst, High Gloss', 'Natural, High Gloss'], dtype=object)

In [474]:
for index,value in enumerate(df['PRICE']):
        if ',' in value:
             df['PRICE'].replace(value, value.replace(',',''), inplace= True)

In [475]:
df['PRICE']=df['PRICE'].astype('float')

In [476]:
df['GUITAR_RANK']=df['GUITAR_RANK'].astype('int')

In [477]:
df['YEARS_WARRANTY']=df['YEARS_WARRANTY'].astype('int')

In [478]:
df['ITEM NUMBER']=df['ITEM NUMBER'].astype('int')

In [479]:
df['NUT WIDTH IN MM'][0]

'43,00 mm'

In [480]:
x=df['NUT WIDTH IN MM'][0]

In [481]:
x.replace(',','').removesuffix(' mm')

'4300'

In [482]:
for index,value in enumerate(df['NUT WIDTH IN MM']):
        if (',' or 'mm') in value:
            value_new=value.replace(',','').removesuffix(' mm')
            df['NUT WIDTH IN MM'].replace(value, value_new, inplace= True)  

In [483]:
df['NUT WIDTH IN MM']=df['NUT WIDTH IN MM'].astype('int')

In [485]:
df['AVAILABLE SINCE']=pd.to_datetime(df['AVAILABLE SINCE'])

In [486]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 99
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MANUFACTURER          98 non-null     object        
 1   MODEL                 98 non-null     object        
 2   PRICE                 98 non-null     float64       
 3   PRODUCT_AVAILABILITY  98 non-null     object        
 4   GUITAR_RANK           98 non-null     int32         
 5   YEARS_WARRANTY        98 non-null     int32         
 6   AVAILABLE SINCE       98 non-null     datetime64[ns]
 7   ITEM NUMBER           98 non-null     int32         
 8   TOP                   98 non-null     object        
 9   BACK AND SIDES        98 non-null     object        
 10  CUTAWAY               98 non-null     object        
 11  FRETBOARD             98 non-null     object        
 12  NUT WIDTH IN MM       98 non-null     int32         
 13  FRETS                 

In [487]:
df.dropna(axis=0, inplace= True)

In [488]:
df.describe()

,PRICE,GUITAR_RANK,YEARS_WARRANTY,ITEM NUMBER,NUT WIDTH IN MM,FRETS
count,98.00000,98.000000,98.0,98.000000,98.000000,98.000000
mean,540.55102,69.397959,3.0,414991.948980,4315.704082,20.030612
std,835.71218,51.879468,0.0,103842.405516,78.255754,0.173150
min,59.00000,3.000000,3.0,152025.000000,4200.000000,20.000000
25%,145.00000,29.250000,3.0,362036.750000,4300.000000,20.000000
50%,297.00000,62.500000,3.0,446055.000000,4300.000000,20.000000
75%,599.00000,100.750000,3.0,483469.000000,4300.000000,20.000000
max,6555.00000,271.000000,3.0,549350.000000,4800.000000,21.000000


In [489]:
df.describe(include='object')

,MANUFACTURER,MODEL,PRODUCT_AVAILABILITY,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,PICKUPS,COLOUR,CASE,INCL. GIGBAG
count,98,98,98,98,98,98,98,98,98,98,98
unique,14,97,2,9,19,2,13,2,7,2,2
top,Harley Benton,D-120CE NT Bundle,In stock,"Spruce, Solid",Mahogany,Yes,Roseacer,Yes,Natural,No,No
freq,31,2,79,47,40,59,18,78,52,83,76


In [490]:
df[['MANUFACTURER','MODEL','PRICE']]

,MANUFACTURER,MODEL,PRICE
0,Fender,CD-60S Black WN,145.0
1,Harley Benton,D-120CE BK,75.0
2,Harley Benton,CLD-60SCE TB,111.0
3,Fender,CD-60 NA V3,111.0
4,Harley Benton,D-120CE NT Bundle,85.0
...,...,...,...
95,Recording King,RD-328 Aged Adirondack,655.0
96,Cort,Earth 100 NS,239.0
97,Guild,D-260CE Deluxe NAT,329.0
98,Harley Benton,CLD-40SM-CE SolidWood,319.0


In [491]:
df[['MANUFACTURER','MODEL','PRICE']].sort_values('PRICE', ascending=False)

,MANUFACTURER,MODEL,PRICE
16,Gibson,Hummingbird Custom Koa,6555.0
57,Gibson,Hummingbird Standard,3299.0
55,Gibson,Slash J-45 Vermillion Burst,2599.0
69,Gibson,Songwriter Cutaway SB,2444.0
68,Gibson,J-45 Standard VS,2190.0
...,...,...,...
36,Harley Benton,D-120CE VS,73.0
35,Harley Benton,D-120BK,66.0
59,Harley Benton,D-120TB,66.0
62,Harley Benton,D-120VS,66.0


In [492]:
df[['MANUFACTURER','MODEL','GUITAR_RANK','PRICE']].sort_values('GUITAR_RANK', ascending=False)

,MANUFACTURER,MODEL,GUITAR_RANK,PRICE
85,Epiphone,J-45 EC Studio EB,271,259.0
80,Epiphone,Masterbilt Texan Faded Cherry,255,577.0
79,Epiphone,USA Texan Antique Natural,188,1899.0
1,Harley Benton,D-120CE BK,177,75.0
84,Fender,CD-140SCE Nat,175,299.0
...,...,...,...,...
43,Harley Benton,D-120NT,7,59.0
67,Harley Benton,D-120CE BK Bundle,6,85.0
44,Fender,CD-60SCE Blk WN,5,229.0
8,Takamine,EF341SC,4,1039.0


In [493]:
df[df['MANUFACTURER']=='Fender']

,MANUFACTURER,MODEL,PRICE,PRODUCT_AVAILABILITY,GUITAR_RANK,YEARS_WARRANTY,AVAILABLE SINCE,ITEM NUMBER,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG
0,Fender,CD-60S Black WN,145.0,In stock,60,3,2019-01-01,455910,"Spruce, Solid",Mahogany,No,Walnut,4300,20,No,Black,No,No
3,Fender,CD-60 NA V3,111.0,In stock,29,3,2006-09-01,190655,Spruce,Mahogany,No,Walnut,4300,20,No,Natural,No,No
20,Fender,FA-125 Blk WN,89.0,In stock,74,3,2019-02-01,460662,Spruce,Mahogany,No,Walnut,4295,20,No,Black,No,Yes
37,Fender,CD-60 NA Bundle,139.0,In stock,102,3,2011-05-01,266387,Spruce,Mahogany,No,Walnut,4300,20,No,Natural,No,Yes
44,Fender,CD-60SCE Blk WN,229.0,In stock,5,3,2018-09-01,447683,"Spruce, Solid",Mahogany,Yes,Walnut,4300,20,Yes,Black,No,No
45,Fender,CD-60S Black WN w/Bag,166.0,In stock,174,3,2022-02-01,538674,"Spruce, Solid",Mahogany,No,Walnut,4300,20,No,Black,No,Yes
46,Fender,CD-140SCE All Mah,299.0,In stock,8,3,2017-02-01,404786,"Mahogany, Solid",Mahogany,Yes,Walnut,4300,20,Yes,Natural,Yes,No
48,Fender,CD-60SCE Nat WN w/Bag,239.0,In stock,40,3,2022-02-01,538648,"Spruce, Solid",Mahogany,Yes,Walnut,4300,20,Yes,Natural,No,Yes
51,Fender,CD-60SCE Nat WN,222.0,In stock,10,3,2019-03-01,461836,"Spruce, Solid",Mahogany,Yes,Walnut,4300,20,Yes,Natural,No,No
63,Fender,CD-60 BK V3,118.0,In stock,11,3,2006-05-01,190654,Spruce,Mahogany,No,Walnut,4300,20,No,Black,No,No


In [497]:
df[ (df['PRICE'] >= 300) & (df['PRICE'] <= 500) ].head()

,MANUFACTURER,MODEL,PRICE,PRODUCT_AVAILABILITY,GUITAR_RANK,YEARS_WARRANTY,AVAILABLE SINCE,ITEM NUMBER,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG
9,Takamine,GD51CE-NAT,389.0,In stock,28,3,2018-10-01,449907,"Spruce, Solid",Walnut,Yes,Laurel,4300,20,Yes,Natural,No,No
13,DAngelico,Premier Bowery ANM,375.0,In stock,70,3,2019-06-01,467202,"Mahogany, Solid",Mahogany,Yes,Ovangkol,4300,20,Yes,Natural Mahogany,No,No
17,Harley Benton,CLD-30SCM-CE SolidWood,319.0,In stock,63,3,2015-01-01,348637,"Cedar, Solid",Solid Mahogany,Yes,Pau Ferro,4450,20,Yes,Natural,No,No
18,Epiphone,Hummingbird Pro Bundle,315.0,In stock,78,3,2012-10-01,299096,Spruce,Mahogany,No,Pau Ferro,4280,20,Yes,Sunburst,No,Yes
19,Takamine,GD51CE-BSB,399.0,In stock,86,3,2018-10-01,444410,"Spruce, Solid",Walnut,Yes,Laurel,4300,20,Yes,Sunburst,No,No


In [498]:
def converter(column):
    if column == 'Yes':
        return 1
    else:
        return 0

In [499]:
def converter2(column):
    if column == 'In stock':
        return 1
    else:
        return 0

In [ ]:
df['CUTAWAY']